In [2]:
!pip install -q kaggle
!pip install bloom-filter

In [47]:
import nltk
from bloom_filter import BloomFilter
import pandas as pd
import sys
import string
import re

Download and unzip Kaggle dataset

In [18]:
 # upload the kaggle.json file provided
 # if doesn't work for some reason see https://www.kaggle.com/general/74235 to create new api key

 from google.colab import files 
 files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ujandeb","key":"9de9f735d28f4c4507a802d6fd74c937"}'}

In [ ]:
# create a directory for the kaggle api key

!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [20]:
!kaggle datasets download -d ranjitha1/hotel-reviews-city-chennai
!unzip /content/hotel-reviews-city-chennai.zip

  0% 0.00/404k [00:00<?, ?B/s]
100% 404k/404k [00:00<00:00, 54.9MB/s]
Archive:  /content/hotel-reviews-city-chennai.zip
  inflating: chennai_reviews.csv     


Loading dataset into pandas dataframe

In [21]:
reviews = pd.read_csv('chennai_reviews.csv')

reviews.head()

,Hotel_name,Review_Title,Review_Text,Sentiment,Rating_Percentage,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Accord Metropolitan,Excellent comfortableness during stay,Its really nice place to stay especially for b...,3,100,NaN,NaN,NaN,NaN
1,Accord Metropolitan,Not too comfortable,It seems that hotel does not check the basic a...,1,20,NaN,NaN,NaN,NaN
2,Accord Metropolitan,NaN,Worst hotel I have ever encountered. I will ne...,1,20,NaN,NaN,NaN,NaN
3,Accord Metropolitan,Best hotel,Had a good time in this hotel and the staff Ku...,3,100,NaN,NaN,NaN,NaN
4,Accord Metropolitan,NaN,good hotel and staff Veg food good non veg bre...,3,100,NaN,NaN,NaN,NaN


Spell Checker for Hotel reviews using Bloom Filters 


Use the list of words provided in NLTK using the following code: 

In [8]:
nltk.download('words') 
 
from nltk.corpus import words 
word_list = words.words() 
print(f'Dictionary length: {len(word_list)}') 
print(word_list[:15])

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
Dictionary length: 236736
['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron', 'Aaronic', 'Aaronical', 'Aaronite', 'Aaronitic', 'Aaru']


 Add each word in the dictionary to a BloomFilter using the following code :

In [9]:
word_filter = BloomFilter(max_elements=len(word_list)) 
 
for word in word_list: 
  word_filter.add(word) 
 
word_set = set(word_list) 

Compare the sizes of the  following data-structures, we have created thus far,  using 
the getsizeof() method. Explain your observations

In [10]:
print('Size of word_list, a Python list containing the English dictionary : {}'.format(sys.getsizeof(word_list)))
print('Size of word_filter, a Bloom filter where we have already added all the words in the English dictionary : {}'.format(sys.getsizeof(word_filter)))
print('Size of word_set, a Python set built from the same list of words in the English dictionary : {}'.format(sys.getsizeof(word_set)))

Size of word_list, a Python list containing the English dictionary : 2115960
Size of word_filter, a Bloom filter where we have already added all the words in the English dictionary : 64
Size of word_set, a Python set built from the same list of words in the English dictionary : 8388840


getsizeof returns the size of an object in bytes. A bloomfilter is simply a bit array. A bit is set to 1 if any of the hash functions maps the input to that bit. As such its size is much less than that of a list or set. A set in python is larger than a list with the same elements becase a set stores a table of hashes of all the elements so it can quickly search and detect duplicate entries.

Compare the time to search for a word in each of the above data structures using the %timeit command

In [17]:
word = 'python'

print('Word list :')
%timeit if word in word_list: pass
print('')

print('Bloom filer :')
%timeit if word in word_filter: pass
print(' ')

print('Word set :')
%timeit if word in word_set: pass

Word list :
1000 loops, best of 5: 1.79 ms per loop

Bloom filer :
100000 loops, best of 5: 10.1 µs per loop
 
Word set :
The slowest run took 30.62 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 59.4 ns per loop


It takes the longest to search a list since each element has to be compared (linear time complexity). Searching a set is the fastest since all elements are hashed and it takes constant time (ammortized) to search an element. Bloom filter does slightly worse since there are multiple hash functions and the algorithm hashes and checks the bit array at all the hashed positions. But it makes up for it with far lower memory usage as seen in the earlier example. 

Go through each review in the dataset and print out the words which do not appear in the dictionary, along with their frequency in the dataset

In [48]:
# lower text and remove punctuation, articles and extra whitespace

def clean_text(s: str) -> str:

  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)

  def white_space_fix(text):
    return ' '.join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

In [56]:
# removing columns that are not required
review_texts = list(reviews['Review_Text'])

# creating a dictionary from all words of the reviews (excluding articles)
review_dict = {}
for review in review_texts:
  if not isinstance(review, str): continue  # ignore if not string
  text = clean_text(review) # cleaning text
  words = text.split()
  for word in words:
    if word in review_dict: # increasing count
      review_dict[word] += 1
    else:
      review_dict[word] = 1 # adding word to dictionary

In [59]:
# iterating over the created dictionary and printing those which are not in the bloom filter

for key, value in review_dict.items():
  if key not in word_filter:
    print('{} : {}'.format(key, value))

seems : 35
amenities : 154
handing : 8
created : 9
changed : 51
encountered : 5
thiis : 4
kumaraishwarya : 4
5 : 98
chennaigood : 4
guys : 34
veg : 44
bathrooms : 70
caulking : 4
restaurants : 125
needed : 56
chefs : 4
meals : 13
advertised : 10
2 : 185
others : 52
nagar : 58
indian : 85
saravana : 11
murugan : 7
welllit : 4
rooms : 1054
mins : 45
checkin : 93
things : 71
increased : 7
enjoyed : 195
residents : 6
located : 356
modes : 4
english : 19
pc : 4
has : 253
internet : 72
tandoori : 8
checkout : 134
4 : 79
stars : 20
luggages : 7
japanese : 9
etc : 82
options : 114
100 : 21
youll : 11
facilities : 342
families : 22
groups : 10
offers : 20
chennai : 593
providers : 4
staffvery : 4
friendlyhelpfulhowever : 4
timetaken : 4
resond : 4
poorbreakfast : 4
betterdaily : 4
soldout : 4
asking : 20
5k : 7
january : 9
2016 : 7
activities : 67
malfunctioned : 4
bandwidth : 4
maxed : 4
evenings : 14
email : 10
45 : 27
stained : 7
curtains : 16
bookings : 10
cards : 4
maintained : 191
staffs 

Write your ideas of how to improve the spell checker. You don’t have to implement  these ideas 

The filter discards proper nouns like Chennai and Oyo. These words could be added to the filter. Different forms of the same root word can be passed through by either reducing the words to their root form or by adding the different forms to the filter itself.